In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd # data manipulation 
import matplotlib # and seaborn for data viz
import seaborn
import sqlalchemy # for sql dbase

In [3]:
#added address
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
pd.set_option('display.max_rows', None) # does not work
pd.set_option('display.max_rows', 60) # does not work 
def fetch_properties(base_url, total_pages, headers):
    data = []  # A list to collect all properties data
    
    for i in range(1, total_pages + 10):  
        url = f"{base_url}{i}"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = bs(response.text, 'html.parser') 
            
            # Find all property cards on the page
            properties = soup.select('div.propertyCard')
            
            for property_card in properties:
                # Extract price
                price = property_card.find('div', class_='propertyCard-priceValue')
                price = price.text.strip() if price else 'N/A'

                # Extract address
                address = property_card.select_one('div.propertyCard-details > a > address')
                address = address.text.strip() if address else 'N/A'

                #Extact Property info 
                bedrooms = property_card.select_one('span.text')  
                bedrooms = bedrooms.text.strip() if bedrooms else 'N/A'
                
                # Append both price and address to the data list
                data.append({'Price': price, 'Address': address, 'Bedrooms': bedrooms})
                
            print(f"Collected {len(properties)} properties from page {i}")  
        else:
            print(f"Failed to retrieve data from page {i}. Status Code:", response.status_code)
    
    return data

# Example Usage
base_url = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E85310&maxPrice=1250000&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='  # Replace with the actual base URL for pagination
total_pages = 10 # Set the total number of pages you expect to scrape -
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

properties = fetch_properties(base_url, total_pages, headers)

# Creating DataFrame
df = pd.DataFrame(properties)
print("\nDataFrame created. Here are some of the properties:")
print(df.head())  

Collected 25 properties from page 1
Collected 25 properties from page 2
Collected 25 properties from page 3
Collected 25 properties from page 4
Collected 25 properties from page 5
Collected 25 properties from page 6
Collected 25 properties from page 7
Collected 25 properties from page 8
Collected 25 properties from page 9
Collected 25 properties from page 10
Collected 25 properties from page 11
Collected 25 properties from page 12
Collected 25 properties from page 13
Collected 25 properties from page 14
Collected 25 properties from page 15
Collected 25 properties from page 16
Collected 25 properties from page 17
Collected 25 properties from page 18
Collected 25 properties from page 19

DataFrame created. Here are some of the properties:
        Price                                 Address Bedrooms
0    £600,000                Forest Road, Walthamstow      N/A
1  £1,200,000  Rectory Road, Walthamstow, London, E17      N/A
2  £1,200,000        Vestry Road, Walthamstow Village      N/A
3

In [4]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

def fetch_properties(base_url, total_pages, headers):
    data = []  # A list to collect all properties data
    
    for i in range(1, total_pages + 1):
        url = f"{base_url}{i}"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = bs(response.text, 'html.parser')
            
            # Find all property cards on the page
            properties = soup.find_all('div', class_='propertyCard')
            
            for property_card in properties:
                # Extract price
                price = property_card.find('div', class_='propertyCard-priceValue')
                price = price.text.strip() if price else 'N/A'

                # Extract address
                address = property_card.select_one('div.propertyCard-details > a > address')
                address = address.text.strip() if address else 'N/A'

                # Extract bedrooms
                bedrooms = property_card.select_one('div.propertyCard-details')
                bedrooms = bedrooms.text.strip() if bedrooms else 'N/A'

                data.append({'Price': price, 'Address': address, 'Bedrooms': bedrooms})
                
            print(f"Collected {len(properties)} properties from page {i}")
        else:
            print(f"Failed to retrieve data from page {i}. Status Code:", response.status_code)
    
    return data



# Example Usage
base_url = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E85310&maxPrice=1250000&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='  # Replace with the actual base URL for pagination
total_pages = 10  # Set the total number of pages you expect to scrape
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

properties = fetch_properties(base_url, total_pages, headers)

# Creating DataFrame
df = pd.DataFrame(properties)
print("\nDataFrame created. Here are some of the properties:")
print(df)  # Display the DataFrame

Collected 25 properties from page 1
Collected 25 properties from page 2
Collected 25 properties from page 3
Collected 25 properties from page 4
Collected 25 properties from page 5
Collected 25 properties from page 6
Collected 25 properties from page 7
Collected 25 properties from page 8
Collected 25 properties from page 9
Collected 25 properties from page 10

DataFrame created. Here are some of the properties:
          Price                                    Address  \
0      £525,000                Chingford Road, Walthamstow   
1    £1,200,000     Rectory Road, Walthamstow, London, E17   
2    £1,200,000           Vestry Road, Walthamstow Village   
3    £1,150,000                   Roland Road, Walthamstow   
4    £1,100,000                 Elmsdale Road, Walthamstow   
..          ...                                        ...   
245    £825,000  Devonshire Road, Walthamstow, London, E17   
246    £800,000   Clifton Avenue, Walthamstow, London, E17   
247    £800,000   Southcote 

In [55]:
df['Price'] = df['Price'].replace('[£,]', '', regex=True).astype(int)
average_price = df['Price'].mean()
print(f"The average house price is: £{average_price:,.2f}")

The average house price is: £916,599.96


In [56]:
df['Price'] = df['Price'].replace('[£,]', '', regex=True).astype(int)
median_price = df['Price'].median()
print(f"The median house price is: £{median_price:,.2f}")

The median house price is: £900,000.00


In [57]:
#London average HP
london_average_price = 523000
df['Percentage Difference'] = ((df['Price'] - london_average_price) / london_average_price) * 100
df['Price'] = df['Price'].apply(lambda x: f"£{x:,.0f}") 
df['Percentage Difference'] = df['Percentage Difference'].apply(lambda x: f"{x:.2f}%")
df

,Price,Address,Bedrooms,Percentage Difference,Percentage Difference_698,Price Difference from £689K,Price Difference from £698K,Leverage,Percentage Difference_768,Percentage Difference_768k,Price Difference from £768K
0,"£525,000","Chingford Road, Walthamstow",2 bedroom terraced house for sale \n\n\...,0.38%,-24.785100,-164000,-173000,-2.43,-31.640625,-31.640625,-243000
1,"£1,200,000","Rectory Road, Walthamstow, London, E17",4 bedroom terraced house for sale \n\n\...,129.45%,71.919771,511000,502000,4.32,56.250000,56.250000,432000
2,"£1,200,000","Vestry Road, Walthamstow Village",3 bedroom detached house for sale \n\n\...,129.45%,71.919771,511000,502000,4.32,56.250000,56.250000,432000
3,"£1,150,000","Roland Road, Walthamstow",5 bedroom house for sale \n\n\n\nRoland...,119.89%,64.756447,461000,452000,3.82,49.739583,49.739583,382000
4,"£1,100,000","Elmsdale Road, Walthamstow",4 bedroom terraced house for sale \n\n\...,110.33%,57.593123,411000,402000,3.32,43.229167,43.229167,332000
...,...,...,...,...,...,...,...,...,...,...,...
245,"£825,000","Devonshire Road, Walthamstow, London, E17",3 bedroom terraced house for sale \n\n\...,57.74%,18.194842,136000,127000,0.57,7.421875,7.421875,57000
246,"£800,000","Clifton Avenue, Walthamstow, London, E17",3 bedroom terraced house for sale \n\n\...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
247,"£800,000","Southcote Road, Walthamstow, London, E17",4 bedroom semi-detached house for sale ...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
248,"£800,000","Browns Road, London, E17",3 bedroom terraced house for sale \n\n\...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000


In [59]:
import pandas as pd

# Assuming df is your DataFrame
# Convert 'Price' column from string to numeric, coercing errors
df['Price'] = pd.to_numeric(df['Price'].replace('[£,]', '', regex=True), errors='coerce') #remember pd.to_numeric


target_price = 768000
df['Percentage Difference_768k'] = ((df['Price'] - target_price) / target_price) * 100

In [60]:
print(df['Price'].head()) 

0     525000
1    1200000
2    1200000
3    1150000
4    1100000
Name: Price, dtype: int64


In [61]:
df['Percentage Difference_768k'] = ((df['Price'] - target_price) / target_price) * 100

In [62]:
print(df[['Price', 'Percentage Difference_768k']].head())

     Price  Percentage Difference_768k
0   525000                  -31.640625
1  1200000                   56.250000
2  1200000                   56.250000
3  1150000                   49.739583
4  1100000                   43.229167


In [63]:
import pandas as pd

# Assuming 'Price' is already a column in df
df['Price'] = pd.to_numeric(df['Price'].replace('[£,]', '', regex=True), errors='coerce')
target_price = 768000
df['Percentage Difference_768k'] = ((df['Price'] - target_price) / target_price) * 100

In [64]:
df

,Price,Address,Bedrooms,Percentage Difference,Percentage Difference_698,Price Difference from £689K,Price Difference from £698K,Leverage,Percentage Difference_768,Percentage Difference_768k,Price Difference from £768K
0,525000,"Chingford Road, Walthamstow",2 bedroom terraced house for sale \n\n\...,0.38%,-24.785100,-164000,-173000,-2.43,-31.640625,-31.640625,-243000
1,1200000,"Rectory Road, Walthamstow, London, E17",4 bedroom terraced house for sale \n\n\...,129.45%,71.919771,511000,502000,4.32,56.250000,56.250000,432000
2,1200000,"Vestry Road, Walthamstow Village",3 bedroom detached house for sale \n\n\...,129.45%,71.919771,511000,502000,4.32,56.250000,56.250000,432000
3,1150000,"Roland Road, Walthamstow",5 bedroom house for sale \n\n\n\nRoland...,119.89%,64.756447,461000,452000,3.82,49.739583,49.739583,382000
4,1100000,"Elmsdale Road, Walthamstow",4 bedroom terraced house for sale \n\n\...,110.33%,57.593123,411000,402000,3.32,43.229167,43.229167,332000
...,...,...,...,...,...,...,...,...,...,...,...
245,825000,"Devonshire Road, Walthamstow, London, E17",3 bedroom terraced house for sale \n\n\...,57.74%,18.194842,136000,127000,0.57,7.421875,7.421875,57000
246,800000,"Clifton Avenue, Walthamstow, London, E17",3 bedroom terraced house for sale \n\n\...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
247,800000,"Southcote Road, Walthamstow, London, E17",4 bedroom semi-detached house for sale ...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
248,800000,"Browns Road, London, E17",3 bedroom terraced house for sale \n\n\...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000


In [65]:
target_price = 768000
df['Price Difference from £768K'] = df['Price'] - target_price

# Display the updated DataFrame
print(df)

       Price                                    Address  \
0     525000                Chingford Road, Walthamstow   
1    1200000     Rectory Road, Walthamstow, London, E17   
2    1200000           Vestry Road, Walthamstow Village   
3    1150000                   Roland Road, Walthamstow   
4    1100000                 Elmsdale Road, Walthamstow   
..       ...                                        ...   
245   825000  Devonshire Road, Walthamstow, London, E17   
246   800000   Clifton Avenue, Walthamstow, London, E17   
247   800000   Southcote Road, Walthamstow, London, E17   
248   800000                   Browns Road, London, E17   
249   800000                    Luton Road, Walthamstow   

                                              Bedrooms Percentage Difference  \
0    2 bedroom terraced house for sale        \n\n\...                 0.38%   
1    4 bedroom terraced house for sale        \n\n\...               129.45%   
2    3 bedroom detached house for sale        \n\n\

In [66]:
df

,Price,Address,Bedrooms,Percentage Difference,Percentage Difference_698,Price Difference from £689K,Price Difference from £698K,Leverage,Percentage Difference_768,Percentage Difference_768k,Price Difference from £768K
0,525000,"Chingford Road, Walthamstow",2 bedroom terraced house for sale \n\n\...,0.38%,-24.785100,-164000,-173000,-2.43,-31.640625,-31.640625,-243000
1,1200000,"Rectory Road, Walthamstow, London, E17",4 bedroom terraced house for sale \n\n\...,129.45%,71.919771,511000,502000,4.32,56.250000,56.250000,432000
2,1200000,"Vestry Road, Walthamstow Village",3 bedroom detached house for sale \n\n\...,129.45%,71.919771,511000,502000,4.32,56.250000,56.250000,432000
3,1150000,"Roland Road, Walthamstow",5 bedroom house for sale \n\n\n\nRoland...,119.89%,64.756447,461000,452000,3.82,49.739583,49.739583,382000
4,1100000,"Elmsdale Road, Walthamstow",4 bedroom terraced house for sale \n\n\...,110.33%,57.593123,411000,402000,3.32,43.229167,43.229167,332000
...,...,...,...,...,...,...,...,...,...,...,...
245,825000,"Devonshire Road, Walthamstow, London, E17",3 bedroom terraced house for sale \n\n\...,57.74%,18.194842,136000,127000,0.57,7.421875,7.421875,57000
246,800000,"Clifton Avenue, Walthamstow, London, E17",3 bedroom terraced house for sale \n\n\...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
247,800000,"Southcote Road, Walthamstow, London, E17",4 bedroom semi-detached house for sale ...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
248,800000,"Browns Road, London, E17",3 bedroom terraced house for sale \n\n\...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000


In [67]:
df['Price Difference from £768K'] = df['Price'] - 768000

LEVERAGE_BASE = 100000
df['Leverage'] = df['Price Difference from £768K'] / LEVERAGE_BASE

# Optional: format the leverage number for readability
df['Leverage'] = df['Leverage'].apply(lambda x: f"{x:.2f}")

print(df)

       Price                                    Address  \
0     525000                Chingford Road, Walthamstow   
1    1200000     Rectory Road, Walthamstow, London, E17   
2    1200000           Vestry Road, Walthamstow Village   
3    1150000                   Roland Road, Walthamstow   
4    1100000                 Elmsdale Road, Walthamstow   
..       ...                                        ...   
245   825000  Devonshire Road, Walthamstow, London, E17   
246   800000   Clifton Avenue, Walthamstow, London, E17   
247   800000   Southcote Road, Walthamstow, London, E17   
248   800000                   Browns Road, London, E17   
249   800000                    Luton Road, Walthamstow   

                                              Bedrooms Percentage Difference  \
0    2 bedroom terraced house for sale        \n\n\...                 0.38%   
1    4 bedroom terraced house for sale        \n\n\...               129.45%   
2    3 bedroom detached house for sale        \n\n\

In [68]:
df

,Price,Address,Bedrooms,Percentage Difference,Percentage Difference_698,Price Difference from £689K,Price Difference from £698K,Leverage,Percentage Difference_768,Percentage Difference_768k,Price Difference from £768K
0,525000,"Chingford Road, Walthamstow",2 bedroom terraced house for sale \n\n\...,0.38%,-24.785100,-164000,-173000,-2.43,-31.640625,-31.640625,-243000
1,1200000,"Rectory Road, Walthamstow, London, E17",4 bedroom terraced house for sale \n\n\...,129.45%,71.919771,511000,502000,4.32,56.250000,56.250000,432000
2,1200000,"Vestry Road, Walthamstow Village",3 bedroom detached house for sale \n\n\...,129.45%,71.919771,511000,502000,4.32,56.250000,56.250000,432000
3,1150000,"Roland Road, Walthamstow",5 bedroom house for sale \n\n\n\nRoland...,119.89%,64.756447,461000,452000,3.82,49.739583,49.739583,382000
4,1100000,"Elmsdale Road, Walthamstow",4 bedroom terraced house for sale \n\n\...,110.33%,57.593123,411000,402000,3.32,43.229167,43.229167,332000
...,...,...,...,...,...,...,...,...,...,...,...
245,825000,"Devonshire Road, Walthamstow, London, E17",3 bedroom terraced house for sale \n\n\...,57.74%,18.194842,136000,127000,0.57,7.421875,7.421875,57000
246,800000,"Clifton Avenue, Walthamstow, London, E17",3 bedroom terraced house for sale \n\n\...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
247,800000,"Southcote Road, Walthamstow, London, E17",4 bedroom semi-detached house for sale ...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
248,800000,"Browns Road, London, E17",3 bedroom terraced house for sale \n\n\...,52.96%,14.613181,111000,102000,0.32,4.166667,4.166667,32000
